# 控制流程

- 以區為單位切割

- 再以頁為單位做切割 取出連結

- 輸入區搜尋=> 翻十頁 或 翻到我過去抓到的檔案最新的資料時停下 開始抓取連結 =>取得連結後開始抓取文章內容 =>存成csv檔

- 為了避免中間抓取文章失敗 當失敗時丟exception 先存檔目前進度 之後再讀檔判斷 回到上一個步驟

In [ ]:
#匯入所需套件

import requests
from bs4 import BeautifulSoup as bs
#from selenium import webdriver
import pandas
import time

In [ ]:
#TOR 用
import requesocks
URL  = 'http://udn.com/news/story/6655/1270657'
session = requesocks.session()
session.proxies = {'http' : 'socks5://127.0.0.1:9050','https': 'socks5://127.0.0.1:9050'}
headers = {
'Host':'search.udn.com',
'Referer':'http://udn.com/search/result/2/'+ '松山區',
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36'
}
res = session.get(URL,headers=headers)
print(res.status_code)
print(res.headers['content-type'])
print(res.content)


In [ ]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
#get all link first
driver = webdriver.Firefox()
#瀏覽器預設等待時間
driver.implicitly_wait(20)

ary = ['中正區','大同區','中山區','松山區','大安區','萬華區','信義區','士林區','北投區','內湖區','南港區','文山區']

#宣告headers 在外避免重複宣告
url = ''
headers={ 'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding':'gzip, deflate, sdch',
        'Accept-Language':'zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4',
        'Connection':'keep-alive',
        'Host':'udn.com',
        'Referer':url,
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36'           
             }
for area in ary:
    for page in range(1,5):
        url = 'http://udn.com/search/result/2/'+ area +'/{}'.format(page)
        headers['Referer'] = url
        print headers['Referer']



In [ ]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
driver = webdriver.Firefox()
#瀏覽器預設等待時間
driver.implicitly_wait(20)
ary = []
links = {}
#從區域開始搜尋
area = '松山區'
count = 1

while True: 
    try:
        url = 'http://udn.com/search/result/1/'+ area
        driver.get(url)
        break
    except:
        print 'try again'


for page in range(1,4):     
    res = driver.page_source
    soup = bs(res)#取得每一頁的連結
    getlink (soup)
    while True:
        try:
            driver.find_element_by_link_text(u"下一頁").click()
        except:
            
        

driver.close()


In [ ]:
#V1
def getlink (soup):
    global count
    for a_tag in soup.select('#search_content a'):
        key = a_tag.get('href')
        ary = [area,page,count]
        links[key] = ary
        count +=1

In [ ]:
i=0
while True:
    try:
        i=i+1
        print 'do'
        if i==6:
            break
        print 4/0
    except:
        print i
        if i==5:
            print 'stop'
            break


In [23]:
#儲存文章連結 正規表示法吃連結
def getlinkv2 (script):
    global count
    m = re.findall('(http://udn.com/news/.*\d+)',script)
    for link in m:
        links = {'link':link,'area':area,'page':page,'count':count}
        linkary.append(links)
        count +=1

In [24]:
#requests 方法 找出網頁特性 發現是jsp.net 直接對伺服器發送要求
import requests
import re
import time
#列出所需查詢區域 一開始先測試3區的效果
#['中正區','大同區','中山區','松山區','大安區','萬華區','信義區','士林區','北投區','內湖區','南港區','文山區']
ary = ['中正區','大同區','中山區','松山區','大安區','萬華區','信義區','士林區','北投區','內湖區','南港區','文山區']
links = {'link':'','area':'','page':'','count':''}
linkary =[]
#每一區
for area in ary:
    count = 1
    headers = {
'Host':'search.udn.com',
'Referer':'http://udn.com/search/result/2/'+ area,
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36'
}
#每一頁    
    for page in range(1,11):
        res =requests.get('http://search.udn.com/search/searchResult4utf8.jsp?ch=udn.news2014&df=2&rc=15&wc=80&pw=220&mc=&q=' + \
                          area + '&fp={}'.format(page),headers = headers,timeout=3)
        time.sleep(3) #避免讓伺服器攻擊
        script = res.text
        getlinkv2(script)
    

In [ ]:
#取出陣列內連結開始連結文章取出內容 標題 時間等

In [25]:
import pandas as pd
data1 = pd.DataFrame(linkary)

In [26]:
data1.to_csv('link.csv',encode='utf-8')

In [3]:
#tor time to gentle haha
import requesocks
session = requesocks.session()
session.proxies = {'http' : 'socks5://127.0.0.1:9050','https': 'socks5://127.0.0.1:9050'}
#requests 方法 找出網頁特性 發現是jsp.net 直接對伺服器發送要求
import requests
import re
import time
import random
#列出所需查詢區域 一開始先測試3區的效果
#['中正區','大同區','中山區','松山區','大安區','萬華區','信義區','士林區','北投區','內湖區','南港區','文山區']
ary = ['中正區','大同區','中山區','松山區','大安區','萬華區','信義區','士林區','北投區','內湖區','南港區','文山區']
links = {'link':'','area':'','page':'','count':''}
linkary =[]
#每一區
for area in ary:
    count = 1
    headers = {
'Host':'search.udn.com',
'Referer':'http://udn.com/search/result/2/'+ area,
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36'
}
#每一頁    
    for page in range(1,11):
        while True:
            try:
                res =session.get('http://search.udn.com/search/searchResult4utf8.jsp?ch=udn.news2014&df=2&rc=15&wc=80&pw=220&mc=&q=' + \
                          area + '&fp={}'.format(page),headers = headers)
                ran = int(random.random()*5)+1
                time.sleep(ran)
                break
            except:
                del session
                session = requesocks.session()
                session.proxies = {'http' : 'socks5://127.0.0.1:9050','https': 'socks5://127.0.0.1:9050'}
                print 'try again random sec'
                ran = int(random.random()*5)+1
                time.sleep(ran)
        #避免讓伺服器察覺攻擊
        script = res.content
        getlinkv2(script)

try again random sec


In [4]:



import pandas as pd
data1 = pd.DataFrame(linkary)

In [27]:
data1.to_csv('link.csv',encode='utf-8')
data1

,area,count,link,page
0,中正區,1,http://udn.com/news/story/7187/1351972,1
1,中正區,2,http://udn.com/news/story/7876/1365348,1
2,中正區,3,http://udn.com/news/story/7185/1355271,1
3,中正區,4,http://udn.com/news/story/7188/1352022,1
4,中正區,5,http://udn.com/news/story/7323/1360653,1
5,中正區,6,http://udn.com/news/story/7328/1357618,1
6,中正區,7,http://udn.com/news/story/6/1355977,1
7,中正區,8,http://udn.com/news/story/7158/1347771,1
8,中正區,9,http://udn.com/news/story/7192/1351848,1
9,中正區,10,http://udn.com/news/story/6886/1355225,1


In [28]:

#get all content
#pcont = article.select('p')
#get['content'] = content(pcont)
def content(pcont):
    cont = ''
    for con in pcont:
        cont = cont + con.text.strip()
    return cont

In [29]:
#tor time to gentle haha
import requesocks
from bs4 import BeautifulSoup as bs
#from selenium import webdriver
import pandas
import time
import random
session = requesocks.session()
session.proxies = {'http' : 'socks5://127.0.0.1:9050','https': 'socks5://127.0.0.1:9050'}

for get in linkary:
    while True:
        ar = get['area']
        url = get['link']
        try:
            headers = {
               'Host':'udn.com',
                'Referer':'http://udn.com/search/result/2/'+ar,
                'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36'
                  }
            res = session.get(url,headers=headers)
            ran = int(random.random()*5)+1
            time.sleep(ran)
            break
        except:
                del session
                session = requesocks.session()
                session.proxies = {'http' : 'socks5://127.0.0.1:9050','https': 'socks5://127.0.0.1:9050'}
                print get['link']
                print 'try again random sec'
                ran = int(random.random()*5)+1
                time.sleep(ran)
    soup = bs(res.content)
    for article in soup.select('#story_body_content'):
        try:
            get['title'] = article.select('h2')[0].text.strip()
            pcont = article.select('p')
            get['content'] = content(pcont) #call the function content
            get['date'] = article.select('h3')[0].text.strip().split(' ')[0]
            get['time'] = article.select('h3')[0].text.strip().split(' ')[1]
        except:
            print 'this link has some problem'
            print get['link'], get['area'], get['count']
            continue


In [30]:
import pandas as pd
data2 = pd.DataFrame(linkary)
data2.to_csv('content2.csv',encoding='utf-8')

In [1]:
import requesocks
from bs4 import BeautifulSoup as bs
#from selenium import webdriver
import pandas
import time
import random
session = requesocks.session()
session.proxies = {'http' : 'socks5://127.0.0.1:9050','https': 'socks5://127.0.0.1:9050'}

url = 'http://udn.com/news/story/3/1327516'
headers = {
               'Host':'udn.com',
                'Referer':'http://udn.com/search/result/2/'+'中正區',
                'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36'
                  }
res = session.get(url,headers=headers)

In [ ]:
import csv
linkary = []
with open('link.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        linkary.append(row)

In [21]:

from bs4 import BeautifulSoup as bs


soup = bs(res.content)
get = {}
for article in soup.select('#story_body_content'):
    get['title'] = article.select('h2')[0].text.strip()
    pcont = article.select('p')
    get['content'] = content(pcont) #call the function content
#     get['date'] = article.select('h3')[0].text.strip().split(' ')[0]
#     get['time'] = article.select('h3')[0].text.strip().split(' ')[1]

print get['content']


為推廣公民參與式預算，北市民政局推出參與式預算公民教育課程。台北市長柯文哲今天也親自現身邀請民眾一同上課，目標培訓2700人，預計首批審議員將於明年4月出爐。柯文哲下午出席「參與式預算培力課程」起跑記者會，他致詞表示，當市長只是將權力拿回來交還給全體市民，這是政治目標，目前參與式預算首步是將預算視覺化上網公布，讓市民清楚看懂預算組成、比較方式。其次，就是教育，他坦言自己也不太清楚參與式預算怎做，所以需要透過民政局推出的公民教育369加嗡嗡嗡的「參與式預算培訓課程」，上課民眾可分別獲得初階卡、進階卡、審議卡與嗡嗡卡，透過在各行政區推廣公民教育，培訓北市千分之一總人口（即2700人）孰悉預算編製，落實全民參與。民政局說明，北市「參與式預算」採漸進式推廣，先讓市民及公部門人員對公民參與觀念正確後，再推動公民提案參與式預算專案。同時，今年5、6、8月已分別辦理4梯次公部門「參與式預算培訓課程」，共計200人參訓成為各局處種子人員；後續還規劃各行政區於12月同步辦理「公民」、「公部門人員」參與式預算教育訓練。民政局自治行政科代理科長王添棟指出，北市中正區已於10月、11月試辦4場次初階課程，每場次名額預定40名、備取10名，結果場場爆滿、成效很好；後續初階課程將開到68場，持續辦理到明年初。中正區完成初階課程民眾黃昱銘表示，自己是中正區相機街商家，北市商圈補助案或專案向來是由北市府專人規劃後再回到商圈，但透過參與式預算則可由真正了解商圈的民眾先提案，再告訴北市府哪些該花和不該花，他個人相當看好此案。民政局說明，公民提案參與式預算推廣課程共分3、6、9小時的初階、進階、審議員課程及審議員實戰培訓等階段，獲得最後「審議卡」者可同時取得擔任工作坊「主持人、桌長、記錄」及提案審議小組「副主持人、記錄」資格。此外，若工作坊及審議實戰訓練滿3場次並通過評鑑者，即可取得專屬個人的記名悠遊卡「嗡嗡卡」，預計首批審議員將於明年4月出爐。民眾凡設籍北市或在北市就學、就業、居住且年滿18歲者，都可報名，12月課程已公告上網，民眾可上公民提案參與式預算資訊平台http://pb.taipei/查詢詳情。


In [ ]:
session = requesocks.session()
session.proxies = {'http' : 'socks5://127.0.0.1:9050','https': 'socks5://127.0.0.1:9050'}
ar = '中正區'
url = 'http://udn.com/news/story/6901/1130200'
headers = {
               'Host':'udn.com',
                'Referer':'http://udn.com/search/result/2/'+ar,
                'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36'
                  }
res = session.get(url,headers=headers)
          
soup  = bs(res.content)    




get={}


In [ ]:


# def content(pcont):
#     global cont
#     for con in pcont:
#         cont = cont + con.text.strip()
#     return cont
cont = ''
for a in soup.select('#story_body_content'):
    ad = a.select('p')
    for con in ad :
        cont = cont + con.text.strip()

In [ ]:
print cont

In [31]:
data2

,area,content,count,date,link,page,time,title
0,中正區,吃飯時間到了，但是沒人陪！沒關係，自己一個人也可以享受美食!其實不少單身的人喜歡自己一個人到...,1,2015-12-09,http://udn.com/news/story/7187/1351972,1,09:01,單身萬歲！5家一個人吃飯也很自在的餐廳
1,中正區,吳碧珠昨率多名北市議員到基隆市議會，基隆市議長宋瑋莉也帶領十多議員相迎，一起推薦郝龍斌，展現...,2,2015-12-09,http://udn.com/news/story/7876/1365348,1,03:23,基隆立委／北市議長 率議員為「郝市長」背書
2,中正區,食我特約美食部落客妮妮要來介紹晴光商圈附近的咖啡店「Lotty café綠蒂咖啡」， Lot...,3,2015-12-08,http://udn.com/news/story/7185/1355271,1,12:20,嚴選烘焙果香咖啡豆 「單品咖啡」香醇好風味
3,中正區,是否和親朋好友聚會早就吃遍大街小巷，不論中式、義式、美式或日式等等通通都嚐鮮過了？那這次就換...,4,2015-12-08,http://udn.com/news/story/7188/1352022,1,07:50,一進門就踏進另個世界！9家必去主題特色餐廳
4,中正區,北市的汙水管接管普及率已達75％，而接管較低的前三名行政區，依序為中正、文山和北投區。位於蛋...,5,2015-12-07,http://udn.com/news/story/7323/1360653,1,07:25,汙水管接管 中正區登錄不全
5,中正區,基隆市府為促進觀光發展，今年在5個區新增5處「借問站」，加上去年的共已有6站，店內提供加水、...,6,2015-12-05,http://udn.com/news/story/7328/1357618,1,08:13,基隆怎麼玩 6處借問站指路
6,中正區,房地合一將上路，不少雙北民眾進場搶搭「輕稅末班車」，其中又以買公寓最熱門，第四季北市市郊、新...,7,2015-12-04,http://udn.com/news/story/6/1355977,1,10:32,搶搭輕稅末班車 十人有三人買公寓
7,中正區,小姪女的暑假作業，是關於文教機構參觀的生活紀錄，愛趴趴造的叔叔終於有藉口，可以計畫這趟基隆海...,8,2015-12-04,http://udn.com/news/story/7158/1347771,1,10:20,比小孩還興奮！雨天超級備案-基隆海科館一日遊
8,中正區,被譽為日本第一天丼「金子半之助」11月初開幕囉，與日本同步推出三款超人氣料理。喜歡日式料理的...,9,2015-12-04,http://udn.com/news/story/7192/1351848,1,09:40,天丼、豬排 選哪一道？ 4家日本人氣新店跨海來台
9,中正區,103學年度大學宿舍供給約有近2成缺口。教育部長吳思華昨表示，教育部已規畫在台北市中正區興建...,10,2015-12-04,http://udn.com/news/story/6886/1355225,1,08:17,6萬大學生沒宿舍 教部籌建大台北跨校宿舍
